In [2]:
!pip3 install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 10.2 MB/s eta 0:00:00


In [3]:
import pymongo
from pymongo import MongoClient
import datetime

In [4]:
client=MongoClient("20.51.211.46", 27000, username="admin", password="abc123")

In [5]:
client.server_info()


{'version': '7.0.0',
 'gitVersion': '37d84072b5c5b9fd723db5fa133fb202ad2317f1',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [7, 0, 0, 0],
 'openssl': {'running': 'OpenSSL 1.1.1f  31 Mar 2020',
  'compiled': 'OpenSSL 1.1.1f  31 Mar 2020'},
 'buildEnvironment': {'distmod': 'ubuntu2004',
  'distarch': 'x86_64',
  'cc': '/opt/mongodbtoolchain/v4/bin/gcc: gcc (GCC) 11.3.0',
  'ccflags': '-Werror -include mongo/platform/basic.h -ffp-contract=off -fasynchronous-unwind-tables -g2 -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -gdwarf-5 -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=sandybridge -mtune=generic -mprefer-vector-width=128 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -gdwarf64 -Wa,--nocompress-debug-sections -fno-builtin-memcmp -Wimplicit-fallthrough=5',
  'cxx': '

gmr database was created on MongoDB

The Google Mobility data was imported into a collection named mobility in the assignment database using the code " mongoimport --db gmr --collection employees --type csv --file employees.csv --host "20.51.211.46:27000" --headerline"

In [6]:
client.list_database_names() #check the list of database available

['admin', 'config', 'gmr', 'local']

In [7]:
# access the database containing the required data
coviddb=client['gmr']
coviddb

Database(MongoClient(host=['20.51.211.46:27000'], document_class=dict, tz_aware=False, connect=True), 'gmr')

In [8]:
# access collection containing all the data
Mobility=coviddb['Mobility']
Mobility

Collection(Database(MongoClient(host=['20.51.211.46:27000'], document_class=dict, tz_aware=False, connect=True), 'gmr'), 'Mobility')

In [ ]:
# Create collection that contains data for only Australia

Mobility = coviddb["Mobility"]
Australia = coviddb["Australia"]

filter_criteria = {"country": "Australia"}

aus_data = Mobility.find(filter_criteria)

Australia.insert_many(aus_data)

In [ ]:
## Remove unwanted fields from Australia collection

fields_to_remove = ['country_region_code', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'] #list of field names to remove

for field in fields_to_remove:
    Australia.update_many({}, {"$unset": {field: 1}}) # Looping through the documents in the collection to remove the specified fields



In [ ]:
# Rename some of the fields

field_rename_mapping = {
    "country_region": "country",
    "sub_region_1": "state",
    "sub_region_2": "region",
    "retail_and_recreation_percent_change_from_baseline": "retail_and_recreation",
    "grocery_and_pharmacy_percent_change_from_baseline": "grocery_and_pharmacy",
    "parks_percent_change_from_baseline": "parks",
    "transit_stations_percent_change_from_baseline": "transit_stations",
    "workplaces_percent_change_from_baseline":"workplaces",
    "residential_percent_change_from_baseline":"residential"
}

for old_field, new_field in field_rename_mapping.items():
    Australia.update_many({}, {"$rename": {old_field: new_field}}) # Looping through the mapping to rename fields



In [ ]:
# convert date to datetime datatype

Australia=coviddb['Australia']

# change the datatype of date to datetime
old_date = "date"

# Update documents to change the datatype of the date field to datetime
Australia.update_many({}, [
    {"$set": {"new_date": {"$toDate": "$" + old_date}}},
    {"$unset":"date"}
])


In [ ]:
Australia=coviddb['Australia']
Australia.update_many({}, {"$unset": {"country": ""}})

In [ ]:
list(Australia.find())

#### Create collection for the cities in each Australian states

In [ ]:
# Create collection that contains only data for Brisbane, Queensland

Australia = coviddb["Australia"]
brs = coviddb["Brisbane"]

filter_criteria = {"state": "Queensland", "region": "Brisbane City"}

aus_data = Australia.find(filter_criteria)

brs.insert_many(aus_data)
brs.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Hobart, Tasmania

Australia = coviddb["Australia"]
hob = coviddb["Hobart"]

filter_criteria = {"state": "Tasmania", "region":"City of Hobart"}

aus_data = Australia.find(filter_criteria)

hob.insert_many(aus_data)
hob.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Perth, WA

Australia = coviddb["Australia"]
pet = coviddb["Perth"]

filter_criteria = {"state": "Western Australia", "region":"City of Perth"}

aus_data = Australia.find(filter_criteria)

pet.insert_many(aus_data)
pet.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Adelaide, Queensland

Australia = coviddb["Australia"]
adl = coviddb["Adelaide"]

filter_criteria = {"state": "South Australia", "region":"Adelaide City Council"}

aus_data = Australia.find(filter_criteria)

adl.insert_many(aus_data)
adl.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Sydney, NSW

Australia = coviddb["Australia"]
syd = coviddb["Sydney"]

filter_criteria = {"state": "New South Wales", "region":"City of Sydney"}

aus_data = Australia.find(filter_criteria)

syd.insert_many(aus_data)
syd.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for ACT

Australia = coviddb["Australia"]
act = coviddb["Australian Capital Territory"]

filter_criteria = {"state": "Australian Capital Territory"}

aus_data = Australia.find(filter_criteria)

act.insert_many(aus_data)
act.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Darwin, Northern Territory

Australia = coviddb["Australia"]
dwn = coviddb["Darwin"]

filter_criteria = {"state": "Northern Territory", "region":"City of Darwin"}

aus_data = Australia.find(filter_criteria)

dwn.insert_many(aus_data)
dwn.update_many({}, {"$unset": {"state": ""}})

In [ ]:
# Create collection that contains only data for Melbourne, Victoria

Australia = coviddb["Australia"]
mel = coviddb["Melbourne"]

filter_criteria = {"state": "Victoria", "region":"City of Melbourne"}

aus_data = Australia.find(filter_criteria)

mel.insert_many(aus_data)
mel.update_many({}, {"$unset": {"state": ""}})

In [9]:
# To view all the created collections

list_of_collections = coviddb.list_collection_names()

# Print the list of collection names
for collection_lists in list_of_collections:
    print(collection_lists)

Melbourne
Sydney
Australian Capital Territory
Perth
Hobart
Darwin
Australia
Brisbane
Mobility
Adelaide
